In [1]:
import pandas as pd
import numpy as np
import importlib
import os
import sys
import argparse
import glob

ROOT = '' # set
sys.path.append(ROOT)

import classical
import utils

# change the width of the cells
from IPython.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

/home/davidsusic/miniconda3/envs/ds_env/lib/python3.8/site-packages/pydub/utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


### Define paths

In [2]:
DATA = utils.check_folder(os.path.join(ROOT, 'data'))
EXPERIMENTS = utils.check_folder(os.path.join(ROOT, 'experiments'))

### Setup parameters

In [3]:
parser = argparse.ArgumentParser(description='PyTorch Training')
parser.add_argument('--dataset', default='PhysioNet(spec)', type=str)
parser.add_argument('--seed_data', default=3, type=int, help='dataset seed when selecting fraction of training set')
parser.add_argument('--valid', default=False, type=bool, help='test model against validation set (when True) or against test set (when False)')
parser.add_argument('--model', default='ResNet', type=str)
parser.add_argument('--method', default='base', type=str)
parser.add_argument('--depth', default=0, type=int)
parser.add_argument('--n_fraction', default=1.0, type=float, help='fraction of train data to be used')
parser.add_argument('--train_balance', default=True, type=bool, help='whether to balance train data')
parser.add_argument('--num_epochs', default=100, type=int)
parser.add_argument('--num_steps', default=100, type=int)
parser.add_argument('--batch_size', default=128, type=int, help='train batchsize')
parser.add_argument('--op', default='adam', type=str, help='optimizer')
parser.add_argument('--lr', default=0.001, type=float, help='initial learning rate')
parser.add_argument('--lr_max', default=0.0025, type=float, help='maximum allowed learning rate')
parser.add_argument('--use_sched', default=True, type=bool, help='whether to use learning rate scheduler')
parser.add_argument('--weight_decay', default=1e-4, type=float, help='weight decay (L2 penalty)')
parser.add_argument('--grad_clip', default=0.1, type=float, help='gradient clipping to prevent exploding gradients')
parser.add_argument('--seed', default=4)
parser.add_argument('--num_classes', default=2, type=int, help='number of classes')
parser.add_argument('--sample_rate', default=1000, type=int, help='signal sample rate')
parser.add_argument('--num_channels', default=4, type=int, help='signal channel number')
parser.add_argument('--sig_len', default=2500, type=int, help='signal length')
parser.add_argument('--latent_space', default=False, type=bool, help='whether to calculate (and plot) latent space hidden features')
parser.add_argument('--classical_space', default=False, type=bool, help='whether to calculate (and plot) classical features')
parser.add_argument('--EXPERIMENTS', default = EXPERIMENTS, type=str, help='path to experiment results')
parser.add_argument('-f') # dummy argument to prevent an error, since argparse is a module designed to parse the arguments passed from the command line
args = parser.parse_args()

### Load dataset

In [4]:
%%time
args.dataset = 'PhysioNet'
args.sig_len = 2500
DATASET = os.path.join(DATA, 'physionet', f'zbytes_physionet_dataset.dat')
dataset = utils.file2dict(DATASET)
print(f'Dataset {DATASET} has been loaded')

Dataset ../../../mnt/eol/Zacasno/davidsusic/CHF/data/physionet/zbytes_physionet_dataset.dat has been loaded
CPU times: user 19.4 s, sys: 10.5 s, total: 29.9 s
Wall time: 45.9 s


In [ ]:
# %%time
# #args.dataset = 'PhysioNet-full'
# DATASET = os.path.join(DATA, 'physionet', f'zbytes_physionet_dataset_full.dat')
# dataset = utils.file2dict(DATASET)
# print(f'Dataset {DATASET} has been loaded')

### Extract features

In [ ]:
importlib.reload(classical)
import warnings
warnings.filterwarnings('ignore')
SAVE_DIR = os.path.join(DATA, 'physionet')
classical.extract_features_python(dataset, SAVE_DIR=SAVE_DIR, start_counter=0)

### Remove segments if their mean envelope is too high/low

In [ ]:
importlib.reload(classical)
FEATURES = os.path.join(DATA, 'physionet', 'features.csv')
SAVE_DIR = os.path.join(DATA, 'physionet')
classical.remove_segments_meanEnvelope(FEATURES, SAVE_DIR, std_factor = 2.0, print_steps = False)

Fill potential missing numbers (sometimes it happens in fourier or welch methods). Some inf values still stay, thus we comment this out and deal with infs and nans in aggregate_features_into_rec_mean_and_sd method.

In [77]:
# FN = 'features_sigma2.0'
# FEATURES = os.path.join(DATA, 'physionet', f'{FN}.csv')
# features = pd.read_csv(FEATURES)
# features.fillna(0, inplace = True)
# FEATURES = os.path.join(DATA, 'physionet', f'{FN}_fillna.csv')
# features.to_csv(FEATURES, index = False)

### Aggregate the features into recordings' mean and SD

In [79]:
importlib.reload(classical)
FEATURES = os.path.join(DATA, 'physionet', 'features_sigma2.0.csv')
SAVE_DIR = os.path.join(DATA, 'physionet')
classical.aggregate_features_into_rec_mean_and_sd(FEATURES, SAVE_DIR, window=6)

AGGREGATING FEATURES INTO MEAN AND SD OF EACH RECORDING
	Feature file ../../../mnt/eol/Zacasno/davidsusic/CHF/data/physionet/features_sigma2.0.csv has been loaded
1: a0058
2: a0059
3: a0060
4: a0061
5: a0062
6: a0063
7: a0064
8: a0065
9: a0066
10: a0067
11: a0072
12: a0073
13: a0074
14: a0075
15: a0076
16: a0077
17: a0078
18: a0079
19: a0082
20: a0083
21: a0084
22: a0087
23: a0089
24: a0090
25: a0092
26: a0095
27: a0096
28: a0097
29: a0098
30: a0099
31: a0100
32: a0101
33: a0103
34: a0104
35: a0107
36: a0110
37: a0111
38: a0112
39: a0113
40: a0114
41: a0115
42: a0116
43: a0117
44: a0119
45: a0120
46: a0121
47: a0122
48: a0123
49: a0124
50: a0126
51: a0128
52: a0130
53: a0131
54: a0132
55: a0133
56: a0134
57: a0135
58: a0137
59: a0138
60: a0140
61: a0141
62: a0142
63: a0143
64: a0144
65: a0145
66: a0146
67: a0147
68: a0148
69: a0149
70: a0150
71: a0151
72: a0152
73: a0153
74: a0154
75: a0155
76: a0156
77: a0157
78: a0158
79: a0159
80: a0160
81: a0161
82: a0162
83: a0163
84: a0164
85: a0

479: b0247
	CUSTOM WARNING: window size 6 must be smaller than the number of segments 6!
480: b0249
481: b0250
482: b0251
483: b0252
484: b0253
	CUSTOM WARNING: window size 6 must be smaller than the number of segments 4!
485: b0254
	CUSTOM WARNING: window size 6 must be smaller than the number of segments 4!
486: b0255
487: b0256
488: b0257
489: b0258
	CUSTOM WARNING: window size 6 must be smaller than the number of segments 6!
490: b0259
	CUSTOM WARNING: window size 6 must be smaller than the number of segments 6!
491: b0260
492: b0261
493: b0262
494: b0263
495: b0264
496: b0265
497: b0266
498: b0267
499: b0268
500: b0269
501: b0270
502: b0271
503: b0272
504: b0273
505: b0274
506: b0275
507: b0276
508: b0277
509: b0278
510: b0279
511: b0280
	CUSTOM WARNING: window size 6 must be smaller than the number of segments 6!
512: b0281
513: b0282
	CUSTOM WARNING: window size 6 must be smaller than the number of segments 6!
514: b0283
515: b0284
516: b0285
517: b0286
	CUSTOM WARNING: window s

811: e00080
812: e00081
813: e00082
814: e00083
815: e00084
816: e00085
817: e00086
818: e00088
819: e00089
820: e00090
821: e00091
822: e00092
823: e00093
824: e00094
825: e00095
826: e00098
827: e00099
828: e00100
829: e00101
830: e00102
831: e00103
832: e00105
833: e00106
834: e00107
835: e00108
836: e00109
837: e00110
838: e00111
839: e00112
840: e00113
841: e00115
842: e00116
843: e00117
844: e00118
845: e00119
846: e00121
847: e00122
848: e00123
849: e00124
850: e00125
851: e00126
852: e00128
853: e00129
854: e00130
855: e00131
856: e00132
857: e00133
858: e00136
859: e00137
860: e00138
861: e00139
862: e00141
863: e00143
864: e00144
865: e00145
866: e00146
867: e00147
868: e00148
869: e00149
870: e00150
871: e00151
872: e00154
873: e00155
874: e00156
875: e00157
876: e00159
877: e00160
878: e00161
879: e00162
880: e00163
881: e00164
882: e00165
883: e00166
884: e00167
885: e00168
886: e00169
887: e00170
888: e00171
889: e00172
890: e00173
891: e00174
	CUSTOM WARNING: window size

1444: e00800
1445: e00801
1446: e00802
1447: e00803
1448: e00804
1449: e00806
1450: e00808
1451: e00809
1452: e00810
1453: e00811
1454: e00812
1455: e00813
1456: e00814
1457: e00815
1458: e00816
1459: e00817
1460: e00818
1461: e00819
1462: e00820
1463: e00821
1464: e00822
1465: e00824
1466: e00825
1467: e00826
1468: e00827
1469: e00828
1470: e00829
1471: e00830
1472: e00831
1473: e00832
1474: e00833
1475: e00834
1476: e00835
1477: e00836
1478: e00837
1479: e00838
1480: e00839
1481: e00840
1482: e00841
1483: e00842
1484: e00843
1485: e00844
1486: e00845
1487: e00846
1488: e00847
1489: e00848
1490: e00849
1491: e00850
1492: e00851
1493: e00852
1494: e00853
1495: e00854
1496: e00855
1497: e00856
1498: e00857
1499: e00858
1500: e00859
1501: e00860
1502: e00861
1503: e00862
1504: e00863
1505: e00864
1506: e00865
1507: e00866
1508: e00867
1509: e00868
1510: e00869
1511: e00870
1512: e00871
1513: e00872
1514: e00874
1515: e00875
1516: e00876
1517: e00877
1518: e00878
1519: e00879
1520: e00880

2057: e01439
2058: e01440
2059: e01441
2060: e01442
2061: e01444
2062: e01445
2063: e01446
2064: e01447
2065: e01448
2066: e01449
2067: e01450
2068: e01451
2069: e01452
2070: e01454
2071: e01455
2072: e01456
2073: e01457
2074: e01458
2075: e01459
2076: e01460
2077: e01461
2078: e01462
2079: e01463
2080: e01464
2081: e01465
2082: e01466
2083: e01467
2084: e01468
2085: e01469
2086: e01470
2087: e01472
2088: e01473
2089: e01474
2090: e01475
2091: e01476
2092: e01477
2093: e01478
2094: e01479
2095: e01480
2096: e01481
2097: e01482
2098: e01483
2099: e01484
2100: e01485
2101: e01486
2102: e01487
2103: e01488
2104: e01489
2105: e01490
2106: e01491
2107: e01492
2108: e01493
2109: e01494
2110: e01495
2111: e01496
2112: e01497
2113: e01498
2114: e01499
2115: e01500
2116: e01501
2117: e01502
2118: e01503
2119: e01504
2120: e01505
2121: e01506
2122: e01507
2123: e01508
2124: e01509
2125: e01510
2126: e01511
2127: e01512
2128: e01513
2129: e01514
2130: e01515
2131: e01516
2132: e01517
2133: e01518

2664: e02063
2665: e02064
2666: e02065
2667: e02066
2668: e02067
2669: e02068
2670: e02069
2671: e02070
2672: e02071
2673: e02072
2674: e02073
2675: e02074
2676: e02075
2677: e02076
2678: e02077
2679: e02078
2680: e02079
2681: e02080
2682: e02081
2683: e02082
2684: e02083
2685: e02084
2686: e02085
2687: e02086
2688: e02087
2689: e02088
2690: e02089
	CUSTOM WARNING: window size 6 must be smaller than the number of segments 4!
2691: e02090
2692: e02092
2693: e02093
2694: e02094
2695: e02095
2696: e02096
2697: e02097
2698: e02098
2699: e02099
2700: e02100
2701: e02101
2702: e02102
2703: e02103
2704: e02104
2705: e02105
2706: e02106
2707: e02107
2708: e02108
2709: e02109
2710: e02110
2711: e02111
2712: e02112
2713: e02113
2714: e02114
2715: e02115
	CUSTOM WARNING: window size 6 must be smaller than the number of segments 6!
2716: e02116
2717: e02117
2718: e02118
2719: e02119
2720: e02120
2721: e02121
	CUSTOM WARNING: window size 6 must be smaller than the number of segments 4!
2722: e02122

#### Create a single vector for each recording

In [29]:
importlib.reload(classical)

FEATURES = os.path.join(DATA, 'physionet', 'features_sigma2.0.csv')
SAVE_DIR = os.path.join(DATA, 'physionet')
classical.aggregate_features_into_rec_mean_and_sd_single_vector(FEATURES, SAVE_DIR)

AGGREGATING FEATURES INTO MEAN AND SD OF EACH RECORDING
	Feature file ../../../mnt/eol/Zacasno/davidsusic/CHF/data/physionet/features_sigma2.0.csv has been loaded
1: a0058
2: a0059
3: a0060
4: a0061
5: a0062
6: a0063
7: a0064
8: a0065
9: a0066
10: a0067
11: a0072
12: a0073
13: a0074
14: a0075
15: a0076
16: a0077
17: a0078
18: a0079
19: a0082
20: a0083
21: a0084
22: a0087
23: a0089
24: a0090
25: a0092
26: a0095
27: a0096
28: a0097
29: a0098
30: a0099
31: a0100
32: a0101
33: a0103
34: a0104
35: a0107
36: a0110
37: a0111
38: a0112
39: a0113
40: a0114
41: a0115
42: a0116
43: a0117
44: a0119
45: a0120
46: a0121
47: a0122
48: a0123
49: a0124
50: a0126
51: a0128
52: a0130
53: a0131
54: a0132
55: a0133
56: a0134
57: a0135
58: a0137
59: a0138
60: a0140
61: a0141
62: a0142
63: a0143
64: a0144
65: a0145
66: a0146
67: a0147
68: a0148
69: a0149
70: a0150
71: a0151
72: a0152
73: a0153
74: a0154
75: a0155
76: a0156
77: a0157
78: a0158
79: a0159
80: a0160
81: a0161
82: a0162
83: a0163
84: a0164
85: a0

741: c0026
742: c0027
743: c0028
744: c0029
745: c0030
746: d0010
747: d0011
748: d0012
749: d0014
750: d0015
751: d0016
752: d0017
753: d0018
754: d0019
755: d0020
756: d0021
757: d0022
758: d0023
759: d0024
760: d0025
761: d0026
762: d0027
763: d0028
764: d0029
765: d0030
766: d0031
767: d0032
768: d0033
769: d0034
770: d0035
771: d0036
772: d0037
773: d0038
774: d0039
775: d0040
776: d0041
777: d0042
778: d0043
779: d0044
780: d0045
781: d0046
782: d0047
783: d0048
784: d0049
785: d0050
786: d0051
787: d0052
788: d0053
789: d0054
790: d0055
791: e00056
792: e00057
793: e00058
794: e00060
795: e00061
796: e00062
797: e00064
798: e00065
799: e00067
800: e00068
801: e00069
802: e00070
803: e00072
804: e00073
805: e00074
806: e00075
807: e00076
808: e00077
809: e00078
810: e00079
811: e00080
812: e00081
813: e00082
814: e00083
815: e00084
816: e00085
817: e00086
818: e00088
819: e00089
820: e00090
821: e00091
822: e00092
823: e00093
824: e00094
825: e00095
826: e00098
827: e00099
828: e

1395: e00748
1396: e00749
1397: e00750
1398: e00751
1399: e00752
1400: e00753
1401: e00754
1402: e00755
1403: e00756
1404: e00757
1405: e00758
1406: e00760
1407: e00761
1408: e00762
1409: e00763
1410: e00764
1411: e00765
1412: e00766
1413: e00767
1414: e00768
1415: e00769
1416: e00770
1417: e00771
1418: e00772
1419: e00773
1420: e00774
1421: e00775
1422: e00778
1423: e00779
1424: e00780
1425: e00781
1426: e00782
1427: e00783
1428: e00784
1429: e00785
1430: e00786
1431: e00787
1432: e00788
1433: e00789
1434: e00790
1435: e00791
1436: e00792
1437: e00793
1438: e00794
1439: e00795
1440: e00796
1441: e00797
1442: e00798
1443: e00799
1444: e00800
1445: e00801
1446: e00802
1447: e00803
1448: e00804
1449: e00806
1450: e00808
1451: e00809
1452: e00810
1453: e00811
1454: e00812
1455: e00813
1456: e00814
1457: e00815
1458: e00816
1459: e00817
1460: e00818
1461: e00819
1462: e00820
1463: e00821
1464: e00822
1465: e00824
1466: e00825
1467: e00826
1468: e00827
1469: e00828
1470: e00829
1471: e00830

2026: e01407
2027: e01408
2028: e01409
2029: e01410
2030: e01411
2031: e01412
2032: e01413
2033: e01414
2034: e01415
2035: e01416
2036: e01417
2037: e01418
2038: e01419
2039: e01420
2040: e01421
2041: e01422
2042: e01423
2043: e01424
2044: e01425
2045: e01426
2046: e01427
2047: e01428
2048: e01429
2049: e01430
2050: e01431
2051: e01433
2052: e01434
2053: e01435
2054: e01436
2055: e01437
2056: e01438
2057: e01439
2058: e01440
2059: e01441
2060: e01442
2061: e01444
2062: e01445
2063: e01446
2064: e01447
2065: e01448
2066: e01449
2067: e01450
2068: e01451
2069: e01452
2070: e01454
2071: e01455
2072: e01456
2073: e01457
2074: e01458
2075: e01459
2076: e01460
2077: e01461
2078: e01462
2079: e01463
2080: e01464
2081: e01465
2082: e01466
2083: e01467
2084: e01468
2085: e01469
2086: e01470
2087: e01472
2088: e01473
2089: e01474
2090: e01475
2091: e01476
2092: e01477
2093: e01478
2094: e01479
2095: e01480
2096: e01481
2097: e01482
2098: e01483
2099: e01484
2100: e01485
2101: e01486
2102: e01487

2657: e02056
2658: e02057
2659: e02058
2660: e02059
2661: e02060
2662: e02061
2663: e02062
2664: e02063
2665: e02064
2666: e02065
2667: e02066
2668: e02067
2669: e02068
2670: e02069
2671: e02070
2672: e02071
2673: e02072
2674: e02073
2675: e02074
2676: e02075
2677: e02076
2678: e02077
2679: e02078
2680: e02079
2681: e02080
2682: e02081
2683: e02082
2684: e02083
2685: e02084
2686: e02085
2687: e02086
2688: e02087
2689: e02088
2690: e02089
2691: e02090
2692: e02092
2693: e02093
2694: e02094
2695: e02095
2696: e02096
2697: e02097
2698: e02098
2699: e02099
2700: e02100
2701: e02101
2702: e02102
2703: e02103
2704: e02104
2705: e02105
2706: e02106
2707: e02107
2708: e02108
2709: e02109
2710: e02110
2711: e02111
2712: e02112
2713: e02113
2714: e02114
2715: e02115
2716: e02116
2717: e02117
2718: e02118
2719: e02119
2720: e02120
2721: e02121
2722: e02122
2723: e02123
2724: e02124
2725: e02125
2726: e02126
2727: e02127
2728: e02128
2729: e02129
2730: e02130
2731: e02131
2732: e02132
2733: e02133

# Experiments

### Find wav subsets for n_fraction < 1.0

#### new seeds

In [5]:
import dataloader_physionet
import argparse

importlib.reload(dataloader_physionet)

NFRAC_WAVS = utils.check_folder(os.path.join(EOL, 'subsets_new'))

n_fractions = [0.015, 0.052, 0.1, 0.2, 0.3, 0.4, 0.6, 0.8, 1.0]

args.train_balance = True
args.dataset = 'PhysioNet'
args.aug = False
args.method = ''
args.valid = False
for n_fraction in n_fractions:
    args.n_fraction = n_fraction
    if n_fraction == 0.015:
        seed_datas = np.arange(1001001, 1001334, 1)
    if n_fraction == 0.052:
        seed_datas = np.arange(1005001, 1005101, 1)
    if n_fraction == 0.1:
        seed_datas = np.arange(1010001, 1010051, 1)
    if n_fraction == 0.2:
        seed_datas = np.arange(1020001, 1020026, 1)
    if n_fraction == 0.3:
        seed_datas = np.arange(1030001, 1030017, 1)
    if n_fraction == 0.4:
        seed_datas = np.arange(1040001, 1040013, 1)
    if n_fraction == 0.6:
        seed_datas = np.arange(1060001, 1060009, 1)
    if n_fraction == 0.8:
        seed_datas = np.arange(1080001, 1080007, 1)
    if n_fraction == 1.0:
        seed_datas = [1100001]
    for seed_data in seed_datas:
        print(f'{seed_data=}')
        args.seed_data = seed_data
    
        FILENAME = os.path.join(NFRAC_WAVS, f'{args.dataset}_seed(data)={args.seed_data}_nfrac={args.n_fraction}_valid={args.valid}.txt')
        FILENAME2 = os.path.join(NFRAC_WAVS, f'{args.dataset}_seed(data)={args.seed_data}_nfrac={args.n_fraction}_valid={args.valid}_num-segs.txt')
        if os.path.exists(FILENAME) and os.path.exists(FILENAME2):
            continue
        data_loader = dataloader_physionet.physionet_dataloader(args, dataset, args.aug)
        train_loader, _ = data_loader.run(mode='train', transform_seed=4)
        sel_wavs = sorted(list(set(train_loader.dataset.train_wav)))
        np.savetxt(FILENAME, sel_wavs, fmt='%s')
        num_segs = len(train_loader.dataset)
        np.savetxt(FILENAME2, [num_segs])
FILENAME = os.path.join(NFRAC_WAVS, f'{args.dataset}_test.txt')
if not os.path.exists(FILENAME):
    data_loader = dataloader_physionet.physionet_dataloader(args, dataset, args.aug)
    test_loader = data_loader.run(mode='test', transform_seed=None)
    sel_wavs = sorted(list(set(test_loader.dataset.test_wav)))
    np.savetxt(FILENAME, sel_wavs, fmt='%s')

/home/davidsusic/miniconda3/envs/ds_env/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


seed_data=1001001
seed_data=1001002
seed_data=1001003
seed_data=1001004
seed_data=1001005
seed_data=1001006
seed_data=1001007
seed_data=1001008
seed_data=1001009
seed_data=1001010
seed_data=1001011
seed_data=1001012
seed_data=1001013
seed_data=1001014
seed_data=1001015
seed_data=1001016
seed_data=1001017
seed_data=1001018
seed_data=1001019
seed_data=1001020
seed_data=1001021
seed_data=1001022
seed_data=1001023
seed_data=1001024
seed_data=1001025
seed_data=1001026
seed_data=1001027
seed_data=1001028
seed_data=1001029
seed_data=1001030
seed_data=1001031
seed_data=1001032
seed_data=1001033
seed_data=1001034
seed_data=1001035
seed_data=1001036
seed_data=1001037
seed_data=1001038
seed_data=1001039
seed_data=1001040
seed_data=1001041
seed_data=1001042
seed_data=1001043
seed_data=1001044
seed_data=1001045
seed_data=1001046
seed_data=1001047
seed_data=1001048
seed_data=1001049
seed_data=1001050
seed_data=1001051
seed_data=1001052
seed_data=1001053
seed_data=1001054
seed_data=1001055
seed_data=

seed_data=1030002
seed_data=1030003
seed_data=1030004
seed_data=1030005
seed_data=1030006
seed_data=1030007
seed_data=1030008
seed_data=1030009
seed_data=1030010
seed_data=1030011
seed_data=1030012
seed_data=1030013
seed_data=1030014
seed_data=1030015
seed_data=1030016
seed_data=1040001
seed_data=1040002
seed_data=1040003
seed_data=1040004
seed_data=1040005
seed_data=1040006
seed_data=1040007
seed_data=1040008
seed_data=1040009
seed_data=1040010
seed_data=1040011
seed_data=1040012
seed_data=1060001
seed_data=1060002
seed_data=1060003
seed_data=1060004
seed_data=1060005
seed_data=1060006
seed_data=1060007
seed_data=1060008
seed_data=1080001
seed_data=1080002
seed_data=1080003
seed_data=1080004
seed_data=1080005
seed_data=1080006
seed_data=1100001


### Run the experiment

In [24]:
importlib.reload(classical)

FEATURES = os.path.join(DATA, 'physionet', f'features_sigma2.0_AggW6.csv')

seed_data = 3
train_balance = True
n_fractions = [0.015, 0.052, 0.1, 0.2, 0.4, 0.6, 0.8, 1.0] 
#n_fractions = [0.015] 
seeds = [1, 2, 3]

for n_fraction in n_fractions:
    for seed in seeds:
        classical.run_experiment(FEATURES = FEATURES, 
                               dataset = 'PhysioNet',
                               n_fraction = n_fraction,
                               keep_only_sd_m_fts = True,
                               majority_vote_prediction = True,
                               tbal=train_balance,
                               seed_data = seed_data,
                               finetune=False,
                               seed=seed,
                               EXPERIMENTS=EXPERIMENTS,
                               show_results=False)

RUNNING CLASSICAL MACHINE LEARNING MODELS EXPERIMENT ON DATASET: PhysioNet
	Features file: ../../../mnt/eol/Zacasno/davidsusic/CHF/data/physionet/features_sigma2.0_AggW6.csv
	n fraction: 0.015
	Seed(data): 3
	Keep only mean and SD features: Enabled
	Majority vote prediction: Enabled
	Finetuning: Disabled
	Seed: 1
	Train balance: Enabled
	Show results: Disabled
Opening the feature file...
Features shape: (83992, 770)
Train length:
	Wavs: 10
	Wavs: ['a0291', 'a0311', 'b0306', 'b0385', 'c0023', 'c0030', 'd0028', 'd0031', 'e00347', 'e00792']
	Segments: 209
Test length:
	Segments: 6922
	Wavs: 297
Non-mean and -SD features were removed
Features shape: (7131, 359)
Selecting features...
Currently working: LogisticRegression
Currently working: DecisionTreeClassifier
Currently working: RandomForestClassifier
Currently working: KNeighborsClassifier
Currently working: GaussianNB
Currently working: SVC
Currently working: SGDClassifier
Currently working: GradientBoostingClassifier
Currently working:

,Classifier,Specificity,Sensitivity,Accuracy,Precision,Recall,F1,ROCAUC
0,LR,0.493151,0.543046,0.518519,0.525641,0.543046,0.534202,0.502586
1,DT,0.69863,0.337748,0.515152,0.536842,0.337748,0.414634,0.507099
2,RF,0.616438,0.562914,0.589226,0.602837,0.562914,0.582192,0.637667
3,KN,0.623288,0.417219,0.518519,0.533898,0.417219,0.468401,0.540597
4,GNB,0.678082,0.523179,0.599327,0.626984,0.523179,0.570397,0.634764
5,SVC,0.671233,0.417219,0.542088,0.567568,0.417219,0.480916,0.557063
6,SGD,0.19863,0.741722,0.474747,0.489083,0.741722,0.589474,0.449356
7,GB,0.684932,0.357616,0.518519,0.54,0.357616,0.430279,0.529257
8,XGB,0.59589,0.496689,0.545455,0.559701,0.496689,0.526316,0.539871


RUNNING CLASSICAL MACHINE LEARNING MODELS EXPERIMENT ON DATASET: PhysioNet
	Features file: ../../../mnt/eol/Zacasno/davidsusic/CHF/data/physionet/features_sigma2.0_AggW6.csv
	n fraction: 0.015
	Seed(data): 3
	Keep only mean and SD features: Enabled
	Majority vote prediction: Enabled
	Finetuning: Disabled
	Seed: 2
	Train balance: Enabled
	Show results: Disabled
Opening the feature file...
Features shape: (83992, 770)
Train length:
	Wavs: 10
	Wavs: ['a0291', 'a0311', 'b0306', 'b0385', 'c0023', 'c0030', 'd0028', 'd0031', 'e00347', 'e00792']
	Segments: 209
Test length:
	Segments: 6922
	Wavs: 297
Non-mean and -SD features were removed
Features shape: (7131, 359)
Selecting features...
Currently working: LogisticRegression
Currently working: DecisionTreeClassifier
Currently working: RandomForestClassifier
Currently working: KNeighborsClassifier
Currently working: GaussianNB
Currently working: SVC
Currently working: SGDClassifier
Currently working: GradientBoostingClassifier
Currently working:

,Classifier,Specificity,Sensitivity,Accuracy,Precision,Recall,F1,ROCAUC
0,LR,0.458904,0.556291,0.508418,0.515337,0.556291,0.535032,0.510977
1,DT,0.726027,0.304636,0.511785,0.534884,0.304636,0.388186,0.499637
2,RF,0.623288,0.602649,0.612795,0.623288,0.602649,0.612795,0.655743
3,KN,0.623288,0.417219,0.518519,0.533898,0.417219,0.468401,0.545655
4,GNB,0.671233,0.536424,0.602694,0.627907,0.536424,0.578571,0.638642
5,SVC,0.691781,0.437086,0.56229,0.594595,0.437086,0.503817,0.56237
6,SGD,0.554795,0.503311,0.52862,0.539007,0.503311,0.520548,0.497936
7,GB,0.609589,0.403974,0.505051,0.516949,0.403974,0.453532,0.530504
8,XGB,0.59589,0.490066,0.542088,0.556391,0.490066,0.521127,0.53597


RUNNING CLASSICAL MACHINE LEARNING MODELS EXPERIMENT ON DATASET: PhysioNet
	Features file: ../../../mnt/eol/Zacasno/davidsusic/CHF/data/physionet/features_sigma2.0_AggW6.csv
	n fraction: 0.015
	Seed(data): 3
	Keep only mean and SD features: Enabled
	Majority vote prediction: Enabled
	Finetuning: Disabled
	Seed: 3
	Train balance: Enabled
	Show results: Disabled
Opening the feature file...
Features shape: (83992, 770)
Train length:
	Wavs: 10
	Wavs: ['a0291', 'a0311', 'b0306', 'b0385', 'c0023', 'c0030', 'd0028', 'd0031', 'e00347', 'e00792']
	Segments: 209
Test length:
	Segments: 6922
	Wavs: 297
Non-mean and -SD features were removed
Features shape: (7131, 359)
Selecting features...
Currently working: LogisticRegression
Currently working: DecisionTreeClassifier
Currently working: RandomForestClassifier
Currently working: KNeighborsClassifier
Currently working: GaussianNB
Currently working: SVC
Currently working: SGDClassifier
Currently working: GradientBoostingClassifier
Currently working:

...

,Classifier,Specificity,Sensitivity,Accuracy,Precision,Recall,F1,ROCAUC
0,LR,0.671233,0.807947,0.740741,0.717647,0.807947,0.760125,0.804182
1,DT,0.691781,0.761589,0.727273,0.71875,0.761589,0.73955,0.776082
2,RF,0.643836,0.827815,0.737374,0.706215,0.827815,0.762195,0.840651
3,KN,0.69863,0.781457,0.740741,0.728395,0.781457,0.753994,0.825615
4,GNB,0.849315,0.490066,0.666667,0.770833,0.490066,0.59919,0.798512
5,SVC,0.712329,0.84106,0.777778,0.751479,0.84106,0.79375,0.850404
6,SGD,0.561644,0.874172,0.720539,0.673469,0.874172,0.760807,0.800032
7,GB,0.671233,0.84106,0.757576,0.725714,0.84106,0.779141,0.848363
8,XGB,0.726027,0.827815,0.777778,0.757576,0.827815,0.791139,0.86324


RUNNING CLASSICAL MACHINE LEARNING MODELS EXPERIMENT ON DATASET: PhysioNet
	Features file: ../../../mnt/eol/Zacasno/davidsusic/CHF/data/physionet/features_sigma2.0_AggW6.csv
	n fraction: 1.0
	Seed(data): 3
	Keep only mean and SD features: Enabled
	Majority vote prediction: Enabled
	Finetuning: Disabled
	Seed: 2
	Train balance: Enabled
	Show results: Disabled
Opening the feature file...
Features shape: (83992, 770)
Train length:
	Wavs: 554
	Wavs: ['a0059', 'a0101', 'a0103', 'a0104', 'a0110', 'a0115', 'a0120', 'a0122', 'a0123', 'a0126', 'a0130', 'a0132', 'a0133', 'a0134', 'a0140', 'a0141', 'a0142', 'a0143', 'a0148', 'a0149', 'a0152', 'a0153', 'a0154', 'a0155', 'a0158', 'a0160', 'a0161', 'a0165', 'a0166', 'a0169', 'a0170', 'a0171', 'a0172', 'a0178', 'a0179', 'a0180', 'a0181', 'a0183', 'a0184', 'a0185', 'a0187', 'a0188', 'a0189', 'a0193', 'a0195', 'a0196', 'a0197', 'a0199', 'a0204', 'a0208', 'a0210', 'a0211', 'a0212', 'a0213', 'a0214', 'a0221', 'a0224', 'a0227', 'a0228', 'a0229', 'a0231', 

,Classifier,Specificity,Sensitivity,Accuracy,Precision,Recall,F1,ROCAUC
0,LR,0.671233,0.807947,0.740741,0.717647,0.807947,0.760125,0.804228
1,DT,0.691781,0.774834,0.734007,0.722222,0.774834,0.747604,0.76574
2,RF,0.643836,0.827815,0.737374,0.706215,0.827815,0.762195,0.840175
3,KN,0.69863,0.781457,0.740741,0.728395,0.781457,0.753994,0.825615
4,GNB,0.849315,0.490066,0.666667,0.770833,0.490066,0.59919,0.798512
5,SVC,0.712329,0.84106,0.777778,0.751479,0.84106,0.79375,0.850358
6,SGD,0.390411,0.97351,0.686869,0.622881,0.97351,0.75969,0.763177
7,GB,0.671233,0.84106,0.757576,0.725714,0.84106,0.779141,0.848136
8,XGB,0.726027,0.827815,0.777778,0.757576,0.827815,0.791139,0.863331


RUNNING CLASSICAL MACHINE LEARNING MODELS EXPERIMENT ON DATASET: PhysioNet
	Features file: ../../../mnt/eol/Zacasno/davidsusic/CHF/data/physionet/features_sigma2.0_AggW6.csv
	n fraction: 1.0
	Seed(data): 3
	Keep only mean and SD features: Enabled
	Majority vote prediction: Enabled
	Finetuning: Disabled
	Seed: 3
	Train balance: Enabled
	Show results: Disabled
Opening the feature file...
Features shape: (83992, 770)
Train length:
	Wavs: 554
	Wavs: ['a0059', 'a0101', 'a0103', 'a0104', 'a0110', 'a0115', 'a0120', 'a0122', 'a0123', 'a0126', 'a0130', 'a0132', 'a0133', 'a0134', 'a0140', 'a0141', 'a0142', 'a0143', 'a0148', 'a0149', 'a0152', 'a0153', 'a0154', 'a0155', 'a0158', 'a0160', 'a0161', 'a0165', 'a0166', 'a0169', 'a0170', 'a0171', 'a0172', 'a0178', 'a0179', 'a0180', 'a0181', 'a0183', 'a0184', 'a0185', 'a0187', 'a0188', 'a0189', 'a0193', 'a0195', 'a0196', 'a0197', 'a0199', 'a0204', 'a0208', 'a0210', 'a0211', 'a0212', 'a0213', 'a0214', 'a0221', 'a0224', 'a0227', 'a0228', 'a0229', 'a0231', 

,Classifier,Specificity,Sensitivity,Accuracy,Precision,Recall,F1,ROCAUC
0,LR,0.636986,0.794702,0.717172,0.693642,0.794702,0.740741,0.80264
1,DT,0.69863,0.735099,0.717172,0.716129,0.735099,0.72549,0.743083
2,RF,0.630137,0.84106,0.737374,0.701657,0.84106,0.76506,0.840969
3,KN,0.69863,0.781457,0.740741,0.728395,0.781457,0.753994,0.825796
4,GNB,0.849315,0.483444,0.6633,0.768421,0.483444,0.593496,0.796607
5,SVC,0.691781,0.847682,0.771044,0.739884,0.847682,0.790123,0.844961
6,SGD,0.773973,0.701987,0.737374,0.76259,0.701987,0.731034,0.817473
7,GB,0.691781,0.834437,0.76431,0.736842,0.834437,0.782609,0.849633
8,XGB,0.691781,0.834437,0.76431,0.736842,0.834437,0.782609,0.85122


# Add classical features extracted from augmented instances into dataframes (same PCG)

In [4]:
args.latent_space=False
args.classical_space=True
args.valid = False
args.num_epochs = 50
args.batch_size = 512
args.lr_max = 0.01

args.dataset = 'UMC'
args.seed_data = 0
args.model='resnet9'
args.aug = False

args.n_fraction = 1.0

seed_datas = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
args.seed = 1

BASE_FEATURES = os.path.join(CHF, 'features', f'(UMC)tableFeatures-Python_raw_filtBandIIR(ZP)4-25-400_normRMS_seg.csv')
base_fts = pd.read_csv(BASE_FEATURES)
base_fts['class'] = base_fts['class'].replace({0:1, 1:0})
base_fts.sort_values(by=['recording', 'segment'], inplace=True) 
#display(base_fts)

cutmix_methods = [
      '(samePCG)durratiomixup+1.0',
      '(samePCG)durmixmagwarp(0.2,4)+1.0',
        ]

for mmm in cutmix_methods:
    args.method = mmm
    print(f'{mmm}')
    for seed_data in seed_datas:
        print(f'{seed_data=}')
        fts = pd.DataFrame()
        args.seed_data = seed_data
        if utils.experiment_already_done(args):
            number = len(glob.glob(os.path.join(utils.experiment_dir(args), 'classical_space', '*')))
            fts = pd.concat([fts, base_fts], ignore_index=True)
            FN = os.path.join(CHF, f'features', f'UMC_augmentation_fts_{mmm}_seed_data={seed_data}_part=0.csv')
            fts.to_csv(FN, index=False)
            #print(number)
            for i in range(number):
                CLASSICAL_FILE = os.path.join(utils.experiment_dir(args), 'classical_space', f'train_{i}.csv')
                fts_batch = pd.read_csv(CLASSICAL_FILE)
                fts_batch.drop(columns=['sig_qual','split'], inplace=True)
                fts_batch = fts_batch.rename(columns={"wav": "recording"})
                fts_batch['recording'] = fts_batch['recording'].apply(lambda x: f"{x}_filtBandIIR(ZP)4-25-400_normRMS")
                fts_batch['recordingName'] = fts_batch['recording'].apply(lambda x: f"{x}.wav")
                fts_batch['patientID'] = fts_batch['recording'].apply(lambda x: f"ID_{x.split('_')[0]}")
                fts_batch['segment'] = 999               
                fts = pd.concat([fts, fts_batch], ignore_index=True)
                fts.sort_values(by=['recording', 'segment'], inplace=True)
                if i%2==1: # here we have batch_size=512 so every 1 epoch equals to two batches
                    FN = os.path.join(CHF, f'features', f'UMC_augmentation_fts_{mmm}_seed_data={seed_data}_part={int((i+1)/2)}.csv')
                    fts.to_csv(FN, index=False)
                    print('Saved to:', FN)
                    #display(fts)

(samePCG)durratiomixup+1.0
seed_data=1
Saved to: ../../../mnt/eol/Zacasno/davidsusic/CHF/features/UMC_augmentation_fts_(samePCG)durratiomixup+1.0_seed_data=1_part=1.csv
Saved to: ../../../mnt/eol/Zacasno/davidsusic/CHF/features/UMC_augmentation_fts_(samePCG)durratiomixup+1.0_seed_data=1_part=2.csv
Saved to: ../../../mnt/eol/Zacasno/davidsusic/CHF/features/UMC_augmentation_fts_(samePCG)durratiomixup+1.0_seed_data=1_part=3.csv
Saved to: ../../../mnt/eol/Zacasno/davidsusic/CHF/features/UMC_augmentation_fts_(samePCG)durratiomixup+1.0_seed_data=1_part=4.csv
Saved to: ../../../mnt/eol/Zacasno/davidsusic/CHF/features/UMC_augmentation_fts_(samePCG)durratiomixup+1.0_seed_data=1_part=5.csv
Saved to: ../../../mnt/eol/Zacasno/davidsusic/CHF/features/UMC_augmentation_fts_(samePCG)durratiomixup+1.0_seed_data=1_part=6.csv
Saved to: ../../../mnt/eol/Zacasno/davidsusic/CHF/features/UMC_augmentation_fts_(samePCG)durratiomixup+1.0_seed_data=1_part=7.csv
Saved to: ../../../mnt/eol/Zacasno/davidsusic/CHF/f

Saved to: ../../../mnt/eol/Zacasno/davidsusic/CHF/features/UMC_augmentation_fts_(samePCG)durratiomixup+1.0_seed_data=2_part=14.csv
Saved to: ../../../mnt/eol/Zacasno/davidsusic/CHF/features/UMC_augmentation_fts_(samePCG)durratiomixup+1.0_seed_data=2_part=15.csv
Saved to: ../../../mnt/eol/Zacasno/davidsusic/CHF/features/UMC_augmentation_fts_(samePCG)durratiomixup+1.0_seed_data=2_part=16.csv
Saved to: ../../../mnt/eol/Zacasno/davidsusic/CHF/features/UMC_augmentation_fts_(samePCG)durratiomixup+1.0_seed_data=2_part=17.csv
Saved to: ../../../mnt/eol/Zacasno/davidsusic/CHF/features/UMC_augmentation_fts_(samePCG)durratiomixup+1.0_seed_data=2_part=18.csv
Saved to: ../../../mnt/eol/Zacasno/davidsusic/CHF/features/UMC_augmentation_fts_(samePCG)durratiomixup+1.0_seed_data=2_part=19.csv
Saved to: ../../../mnt/eol/Zacasno/davidsusic/CHF/features/UMC_augmentation_fts_(samePCG)durratiomixup+1.0_seed_data=2_part=20.csv
Saved to: ../../../mnt/eol/Zacasno/davidsusic/CHF/features/UMC_augmentation_fts_(sa

Saved to: ../../../mnt/eol/Zacasno/davidsusic/CHF/features/UMC_augmentation_fts_(samePCG)durratiomixup+1.0_seed_data=3_part=27.csv
Saved to: ../../../mnt/eol/Zacasno/davidsusic/CHF/features/UMC_augmentation_fts_(samePCG)durratiomixup+1.0_seed_data=3_part=28.csv
Saved to: ../../../mnt/eol/Zacasno/davidsusic/CHF/features/UMC_augmentation_fts_(samePCG)durratiomixup+1.0_seed_data=3_part=29.csv
Saved to: ../../../mnt/eol/Zacasno/davidsusic/CHF/features/UMC_augmentation_fts_(samePCG)durratiomixup+1.0_seed_data=3_part=30.csv
Saved to: ../../../mnt/eol/Zacasno/davidsusic/CHF/features/UMC_augmentation_fts_(samePCG)durratiomixup+1.0_seed_data=3_part=31.csv
Saved to: ../../../mnt/eol/Zacasno/davidsusic/CHF/features/UMC_augmentation_fts_(samePCG)durratiomixup+1.0_seed_data=3_part=32.csv
Saved to: ../../../mnt/eol/Zacasno/davidsusic/CHF/features/UMC_augmentation_fts_(samePCG)durratiomixup+1.0_seed_data=3_part=33.csv
Saved to: ../../../mnt/eol/Zacasno/davidsusic/CHF/features/UMC_augmentation_fts_(sa

Saved to: ../../../mnt/eol/Zacasno/davidsusic/CHF/features/UMC_augmentation_fts_(samePCG)durratiomixup+1.0_seed_data=4_part=40.csv
Saved to: ../../../mnt/eol/Zacasno/davidsusic/CHF/features/UMC_augmentation_fts_(samePCG)durratiomixup+1.0_seed_data=4_part=41.csv
Saved to: ../../../mnt/eol/Zacasno/davidsusic/CHF/features/UMC_augmentation_fts_(samePCG)durratiomixup+1.0_seed_data=4_part=42.csv
Saved to: ../../../mnt/eol/Zacasno/davidsusic/CHF/features/UMC_augmentation_fts_(samePCG)durratiomixup+1.0_seed_data=4_part=43.csv
Saved to: ../../../mnt/eol/Zacasno/davidsusic/CHF/features/UMC_augmentation_fts_(samePCG)durratiomixup+1.0_seed_data=4_part=44.csv
Saved to: ../../../mnt/eol/Zacasno/davidsusic/CHF/features/UMC_augmentation_fts_(samePCG)durratiomixup+1.0_seed_data=4_part=45.csv
Saved to: ../../../mnt/eol/Zacasno/davidsusic/CHF/features/UMC_augmentation_fts_(samePCG)durratiomixup+1.0_seed_data=4_part=46.csv
Saved to: ../../../mnt/eol/Zacasno/davidsusic/CHF/features/UMC_augmentation_fts_(sa

Saved to: ../../../mnt/eol/Zacasno/davidsusic/CHF/features/UMC_augmentation_fts_(samePCG)durratiomixup+1.0_seed_data=6_part=3.csv
Saved to: ../../../mnt/eol/Zacasno/davidsusic/CHF/features/UMC_augmentation_fts_(samePCG)durratiomixup+1.0_seed_data=6_part=4.csv
Saved to: ../../../mnt/eol/Zacasno/davidsusic/CHF/features/UMC_augmentation_fts_(samePCG)durratiomixup+1.0_seed_data=6_part=5.csv
Saved to: ../../../mnt/eol/Zacasno/davidsusic/CHF/features/UMC_augmentation_fts_(samePCG)durratiomixup+1.0_seed_data=6_part=6.csv
Saved to: ../../../mnt/eol/Zacasno/davidsusic/CHF/features/UMC_augmentation_fts_(samePCG)durratiomixup+1.0_seed_data=6_part=7.csv
Saved to: ../../../mnt/eol/Zacasno/davidsusic/CHF/features/UMC_augmentation_fts_(samePCG)durratiomixup+1.0_seed_data=6_part=8.csv
Saved to: ../../../mnt/eol/Zacasno/davidsusic/CHF/features/UMC_augmentation_fts_(samePCG)durratiomixup+1.0_seed_data=6_part=9.csv
Saved to: ../../../mnt/eol/Zacasno/davidsusic/CHF/features/UMC_augmentation_fts_(samePCG)d

...

Saved to: ../../../mnt/eol/Zacasno/davidsusic/CHF/features/UMC_augmentation_fts_(samePCG)durmixmagwarp(0.2,4)+1.0_seed_data=5_part=44.csv
Saved to: ../../../mnt/eol/Zacasno/davidsusic/CHF/features/UMC_augmentation_fts_(samePCG)durmixmagwarp(0.2,4)+1.0_seed_data=5_part=45.csv
Saved to: ../../../mnt/eol/Zacasno/davidsusic/CHF/features/UMC_augmentation_fts_(samePCG)durmixmagwarp(0.2,4)+1.0_seed_data=5_part=46.csv
Saved to: ../../../mnt/eol/Zacasno/davidsusic/CHF/features/UMC_augmentation_fts_(samePCG)durmixmagwarp(0.2,4)+1.0_seed_data=5_part=47.csv
Saved to: ../../../mnt/eol/Zacasno/davidsusic/CHF/features/UMC_augmentation_fts_(samePCG)durmixmagwarp(0.2,4)+1.0_seed_data=5_part=48.csv
Saved to: ../../../mnt/eol/Zacasno/davidsusic/CHF/features/UMC_augmentation_fts_(samePCG)durmixmagwarp(0.2,4)+1.0_seed_data=5_part=49.csv
Saved to: ../../../mnt/eol/Zacasno/davidsusic/CHF/features/UMC_augmentation_fts_(samePCG)durmixmagwarp(0.2,4)+1.0_seed_data=5_part=50.csv
seed_data=6
Saved to: ../../../mnt

Saved to: ../../../mnt/eol/Zacasno/davidsusic/CHF/features/UMC_augmentation_fts_(samePCG)durmixmagwarp(0.2,4)+1.0_seed_data=7_part=4.csv
Saved to: ../../../mnt/eol/Zacasno/davidsusic/CHF/features/UMC_augmentation_fts_(samePCG)durmixmagwarp(0.2,4)+1.0_seed_data=7_part=5.csv
Saved to: ../../../mnt/eol/Zacasno/davidsusic/CHF/features/UMC_augmentation_fts_(samePCG)durmixmagwarp(0.2,4)+1.0_seed_data=7_part=6.csv
Saved to: ../../../mnt/eol/Zacasno/davidsusic/CHF/features/UMC_augmentation_fts_(samePCG)durmixmagwarp(0.2,4)+1.0_seed_data=7_part=7.csv
Saved to: ../../../mnt/eol/Zacasno/davidsusic/CHF/features/UMC_augmentation_fts_(samePCG)durmixmagwarp(0.2,4)+1.0_seed_data=7_part=8.csv
Saved to: ../../../mnt/eol/Zacasno/davidsusic/CHF/features/UMC_augmentation_fts_(samePCG)durmixmagwarp(0.2,4)+1.0_seed_data=7_part=9.csv
Saved to: ../../../mnt/eol/Zacasno/davidsusic/CHF/features/UMC_augmentation_fts_(samePCG)durmixmagwarp(0.2,4)+1.0_seed_data=7_part=10.csv
Saved to: ../../../mnt/eol/Zacasno/david

Saved to: ../../../mnt/eol/Zacasno/davidsusic/CHF/features/UMC_augmentation_fts_(samePCG)durmixmagwarp(0.2,4)+1.0_seed_data=8_part=14.csv
Saved to: ../../../mnt/eol/Zacasno/davidsusic/CHF/features/UMC_augmentation_fts_(samePCG)durmixmagwarp(0.2,4)+1.0_seed_data=8_part=15.csv
Saved to: ../../../mnt/eol/Zacasno/davidsusic/CHF/features/UMC_augmentation_fts_(samePCG)durmixmagwarp(0.2,4)+1.0_seed_data=8_part=16.csv
Saved to: ../../../mnt/eol/Zacasno/davidsusic/CHF/features/UMC_augmentation_fts_(samePCG)durmixmagwarp(0.2,4)+1.0_seed_data=8_part=17.csv
Saved to: ../../../mnt/eol/Zacasno/davidsusic/CHF/features/UMC_augmentation_fts_(samePCG)durmixmagwarp(0.2,4)+1.0_seed_data=8_part=18.csv
Saved to: ../../../mnt/eol/Zacasno/davidsusic/CHF/features/UMC_augmentation_fts_(samePCG)durmixmagwarp(0.2,4)+1.0_seed_data=8_part=19.csv
Saved to: ../../../mnt/eol/Zacasno/davidsusic/CHF/features/UMC_augmentation_fts_(samePCG)durmixmagwarp(0.2,4)+1.0_seed_data=8_part=20.csv
Saved to: ../../../mnt/eol/Zacasno

Saved to: ../../../mnt/eol/Zacasno/davidsusic/CHF/features/UMC_augmentation_fts_(samePCG)durmixmagwarp(0.2,4)+1.0_seed_data=9_part=24.csv
Saved to: ../../../mnt/eol/Zacasno/davidsusic/CHF/features/UMC_augmentation_fts_(samePCG)durmixmagwarp(0.2,4)+1.0_seed_data=9_part=25.csv
Saved to: ../../../mnt/eol/Zacasno/davidsusic/CHF/features/UMC_augmentation_fts_(samePCG)durmixmagwarp(0.2,4)+1.0_seed_data=9_part=26.csv
Saved to: ../../../mnt/eol/Zacasno/davidsusic/CHF/features/UMC_augmentation_fts_(samePCG)durmixmagwarp(0.2,4)+1.0_seed_data=9_part=27.csv
Saved to: ../../../mnt/eol/Zacasno/davidsusic/CHF/features/UMC_augmentation_fts_(samePCG)durmixmagwarp(0.2,4)+1.0_seed_data=9_part=28.csv
Saved to: ../../../mnt/eol/Zacasno/davidsusic/CHF/features/UMC_augmentation_fts_(samePCG)durmixmagwarp(0.2,4)+1.0_seed_data=9_part=29.csv
Saved to: ../../../mnt/eol/Zacasno/davidsusic/CHF/features/UMC_augmentation_fts_(samePCG)durmixmagwarp(0.2,4)+1.0_seed_data=9_part=30.csv
Saved to: ../../../mnt/eol/Zacasno

Saved to: ../../../mnt/eol/Zacasno/davidsusic/CHF/features/UMC_augmentation_fts_(samePCG)durmixmagwarp(0.2,4)+1.0_seed_data=10_part=34.csv
Saved to: ../../../mnt/eol/Zacasno/davidsusic/CHF/features/UMC_augmentation_fts_(samePCG)durmixmagwarp(0.2,4)+1.0_seed_data=10_part=35.csv
Saved to: ../../../mnt/eol/Zacasno/davidsusic/CHF/features/UMC_augmentation_fts_(samePCG)durmixmagwarp(0.2,4)+1.0_seed_data=10_part=36.csv
Saved to: ../../../mnt/eol/Zacasno/davidsusic/CHF/features/UMC_augmentation_fts_(samePCG)durmixmagwarp(0.2,4)+1.0_seed_data=10_part=37.csv
Saved to: ../../../mnt/eol/Zacasno/davidsusic/CHF/features/UMC_augmentation_fts_(samePCG)durmixmagwarp(0.2,4)+1.0_seed_data=10_part=38.csv
Saved to: ../../../mnt/eol/Zacasno/davidsusic/CHF/features/UMC_augmentation_fts_(samePCG)durmixmagwarp(0.2,4)+1.0_seed_data=10_part=39.csv
Saved to: ../../../mnt/eol/Zacasno/davidsusic/CHF/features/UMC_augmentation_fts_(samePCG)durmixmagwarp(0.2,4)+1.0_seed_data=10_part=40.csv
Saved to: ../../../mnt/eol/

# Add classical features extracted from augmented instances into dataframes

In [19]:
args.latent_space=False
args.classical_space=True
args.valid = False
args.num_epochs = 50
args.batch_size = 64
args.lr_max = 0.01

args.dataset = 'UMC'
args.seed_data = 0
args.model='resnet9'
args.aug = False

args.n_fraction = 1.0

seed_datas = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
args.seed = 1

cutmix_methods = [
      'base',
      'durratiomixup+1.0',
      'durmixmagwarp(0.2,4)+1.0',
        ]

for mmm in cutmix_methods:
    args.method = mmm
    print(f'{mmm}')
    for seed_data in seed_datas:
        print(f'{seed_data=}')
        fts = pd.DataFrame()
        args.seed_data = seed_data
        if utils.experiment_already_done(args):
            number = len(glob.glob(os.path.join(utils.experiment_dir(args), 'classical_space', '*')))
            for i in range(number):
                CLASSICAL_FILE = os.path.join(utils.experiment_dir(args), 'classical_space', f'train_{i}.csv')
                fts_batch = pd.read_csv(CLASSICAL_FILE)
                fts = pd.concat([fts, fts_batch], ignore_index=True)
                #print(i, len(fts_batch), len(fts))
        FN = os.path.join(CHF, f'features', f'UMC_augmentation_fts_{mmm}_seed_data={seed_data}.csv')
        fts.to_csv(FN, index=False)
        print('Saved to:', FN)

durratiomixup+1.0
seed_data=1
Saved to: ../../../mnt/eol/Zacasno/davidsusic/CHF/features/UMC_augmentation_fts_durratiomixup+1.0_seed_data=1.csv
seed_data=2
Saved to: ../../../mnt/eol/Zacasno/davidsusic/CHF/features/UMC_augmentation_fts_durratiomixup+1.0_seed_data=2.csv
seed_data=3
Saved to: ../../../mnt/eol/Zacasno/davidsusic/CHF/features/UMC_augmentation_fts_durratiomixup+1.0_seed_data=3.csv
seed_data=4
Saved to: ../../../mnt/eol/Zacasno/davidsusic/CHF/features/UMC_augmentation_fts_durratiomixup+1.0_seed_data=4.csv
seed_data=5
Saved to: ../../../mnt/eol/Zacasno/davidsusic/CHF/features/UMC_augmentation_fts_durratiomixup+1.0_seed_data=5.csv
seed_data=6
Saved to: ../../../mnt/eol/Zacasno/davidsusic/CHF/features/UMC_augmentation_fts_durratiomixup+1.0_seed_data=6.csv
seed_data=7
Saved to: ../../../mnt/eol/Zacasno/davidsusic/CHF/features/UMC_augmentation_fts_durratiomixup+1.0_seed_data=7.csv
seed_data=8
Saved to: ../../../mnt/eol/Zacasno/davidsusic/CHF/features/UMC_augmentation_fts_durratio